# Blackjack game

This notebook was an exercise of a Python course. The task was to program a Blackjack game. It's problably a standard practice for Object-Oriented Programming, but then the teacher is based in Las Vegas. I went a bit beyond the requirements and added more than one player, which made things considerably more messy.

First I import modules and define some list: Card ranks, suits and a dictionary with the values.

In [ ]:
import random
from IPython.display import clear_output

suits = ['Hearts', 'Spades', 'Diamonds', 'Clubs']
ranks = ['Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace']
values = {'Two':2,'Three':3,'Four':4,'Five':5,'Six':6,'Seven':7,'Eight':8,'Nine':9,'Ten':10,'Jack':10,'Queen':10,'King':10,'Ace':11} # dict

Now I define a Card class, pretty straightforward, with suit and rank:

In [ ]:
class Card():
    
    def __init__(self,suit,rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
    
    def __str__(self):
        return f"{self.rank} of {self.suit}"

Next comes a Deck class, with functions to shuffle and draw one, which gets removed from the deck. I don't want print to print out all cards, just the number, but I defined a separate function to print all out explicitly (mainly for debugging).

In [ ]:
class Deck():
    
    def __init__(self):
        self.all_cards = []
        for suit in suits:
            for rank in ranks:
                self.all_cards.append(Card(suit,rank))
        self.num_cards = len(self.all_cards)        
        
    def __str__(self):
        return f"Deck contains {self.num_cards} cards"
    
    def shuffle(self):
        random.shuffle(self.all_cards)
        
    def printall(self):
        for card in self.all_cards:
            print(card)
    
    def deal_one(self):
        return self.all_cards.pop()

Next comes the player class. This seems quite overloaded and can probably be done more elegantly. The player has a hand (list) of cards, a number of chips, current bet. 

In [ ]:
class Player():
    def __init__(self,name,chips):
        self.name = name
        self.chips = chips
        self.all_cards = []
        self.total_value = 0
        self.bet = 0
        self.out = False
        self.bust = False
        self.ace_number = 0

#Every time a card is added to the player, the total value of the hand is updated, taking into account aces        
    def add_card(self,card):
        if card.rank == "Ace":
            self.ace_number += 1
        self.all_cards.append(card)
        self.total_value += card.value
        while self.total_value > 21 and self.ace_number > 0:
            self.ace_number -= 1
            self.total_value -= 10
            
#Betting too much automatically bets all remaining chips. Why not?
    def place_bet(self,amount):
        if amount > self.chips:
            print(f"You only have {self.chips} chips. Maximum amount placed.")
            self.chips = 0
            self.bet = self.chips
        else:    
            self.chips -= amount
            self.bet += amount
        
    def add_chips(self,amount):
        self.chips += amount

    #This explicitly displays all cards the player currently has    
    def on_table(self):
        print(f"{self.name} has: ")
        for card in self.all_cards:
            print(f"{card.rank} of {card.suit}")
        print("")
    
    #To reset the player: Remove all cards, reset boolean flags
    def reset(self):
        self.all_cards = []
        self.total_value = 0
        self.out = False
        self.bust = False
        self.ace_number = 0
                    
    def __str__(self):
        return f"{self.name} has {self.chips} chips."
    


I made the dealer a subclass of the player? Smart or stupid?

In [ ]:
class Dealer(Player):
    def __init__(self,hit_until=17):
        self.name = "Dealer"
        self.chips = 0
        self.hit_until = hit_until
        self.all_cards = []
        self.total_value = 0
    
    #When the dealer's hand is displayed, one card is hidden
    def on_table(self):
        print(f"{self.name} has: ")
        print("1 face-down card")
        for card in self.all_cards[1::]:
            print(f"{card.rank} of {card.suit}")
        print("")
    #But later, all cards are revealed
    def reveal(self):
        print(f"{self.name} has: ")
        for card in self.all_cards:
            print(f"{card.rank} of {card.suit}")
        print("")
   


Now the players get generated, first ask how many (max. 4 is completely arbitrary), then generate them. By default, all players are named Ollie, after my cat.

In [ ]:
def numplayers():
    numstr = '0'
    while numstr not in ('1','2','3','4'):
        numstr = input("How many players want to play? (1 - 4)") or '1'
    return int(numstr)

def init_players(n_players=1):
    players = []

    #The or statements below trigger when the user just presses enter, as empty strings count as False in Python
    for n in range(n_players):
        name = input (f"Enter your name, player {n+1}: ") or f'Ollie {n+1}'
        chips = 'A'
        while not chips.isdigit():
            chips = input (f"How many chips do you have, {name}? ") or '100'
        players.append(Player(name,int(chips)))
    print("")
    return players

Now I define some helpful functions that display information about chips, bets, etc.

In [ ]:
#Show all chips of all players, called at the beginning of each game
def showchips():
    global players
    for p in players:
        if type(p) != Dealer:
            print(p)
    print()

#Show all bets made by all players    
def showbets():
    global players
    for p in players:
        if type(p) != Dealer:
            print(f"{p.name} has bet {p.bet} chips.")
    print()

#Show all cards on all players
def showcards():
    global players
    clear_output()
    for p in players:
        p.on_table()
        
#This gets called when the game is over, to give an overview of the final situation        
def showfinal():
    global players
    clear_output()
    dealer.reveal()
    if dealer.total_value > 21:
        print(f"Value: {dealer.total_value} :BUSTED!\n\n")
    else:
        print(f"Value: {dealer.total_value}\n\n")
    
    
    for p in players[1:]:
        p.on_table()
        if p.total_value > 21:
            print(f"Value: {p.total_value} :BUSTED!\n\n")
        else:
            print(f"Value: {p.total_value}\n\n")
    

Below is called before the hitting/standing etc. starts, to see if someone has blackjack at the start (and then resolve the game immediately). Not sure if this is the actual rules...

In [ ]:
def checkbj():
    global players
    bj = False
    #If the dealer has blackjack, everyone loses, unless they also have blackjack
    #If you lose, the bet gets removed. The chips do not get removed, because that happens during betting
    if dealer.total_value == 21:
        bj = True
        print("Dealer has blackjack!")
        for p in players[1:]:
            if p.total_value == 21:
                print(f"{p.name} draws.")
                p.add_chips(p.bet)
            else:
                print(f"{p.name} loses {p.bet} chips.")
            p.bet = 0
    #If the dealer does not have blackjack, check all players
    else:
        for p in players[1:]:
            if p.total_value == 21:
                print(f"{p.name} has blackjack!")
                print(f"{p.name} wins {3*p.bet//2} chips!")
                #The original bet gets returned + 1.5 times the bet is won
                p.add_chips(5*p.bet//2)
                bj = True
    #bj remains False, unless someone had blackjack            
    return bj

The next function is the roundup to check who wins, in case there was no blackjack

In [ ]:
def roundup():
    global players
    for p in players[1:]:
        #If the dealer busts, everyone wins, unless they busted too
        if dealer.bust:
            if not p.bust:
                print(f"{p.name} wins {p.bet} chips!")
                p.add_chips(2*p.bet)
            else:
                print(f"{p.name} loses {p.bet} chips!")
        else:
        #If the dealer does not bust, check each player for win/loss/draw/bust and adjust chips
            if not p.bust:
                if p.total_value > dealer.total_value:
                    print(f"{p.name} wins {p.bet} chips!")
                    p.add_chips(2*p.bet)
                elif p.total_value == dealer.total_value:
                    print(f"Draw between {p.name} and Dealer!")
                    p.add_chips(p.bet)
                else:
                    print(f"{p.name} loses {p.bet} chips!")
            else:
                print(f"{p.name} loses {p.bet} chips!")
        p.bet = 0

The ugly monstrosity below is the game itself. But it works.

In [ ]:
#Initialize the program, get number, name and chips of players
game = 1
n_players = numplayers()
players = init_players(n_players)
dealer = Dealer(17)
#The dealer is player number 0
players.insert(0,dealer)

#This while true loop runs until all players are bankrupt or the user doesn't want to play anymore
while True:
    #Initialize the current game: Get a new deck with all cards
    clear_output()
    print(f"Dealer must hit until {dealer.hit_until}, then stay.\n")
    showchips()
    deck = Deck()
    deck.shuffle()
    
    #Loop through all players (but not the dealer) to get them to bet
    print(f"Game {game}\n")
    for p in players[1:]:
        bet = 'A'
        while not bet.isdigit():
            bet = input(f"Place your bet, {p.name}!\n")
        p.place_bet(int(bet))
        
    round = 1
    showbets()
    
    #Everyone gets two cards from the deck (including the dealer)
    for p in players:
        p.reset()
        p.add_card(deck.deal_one())
        p.add_card(deck.deal_one())
        p.on_table()

    
    bj = False
    n_out = 0
    
    #Now continously loop through all players until all are out (stand, have busted, have surrendered)
    while n_out != len(players)-1:
        #Check if there is an immediate blackjack. In that case, the game is already over
        if round == 1:
            bj = checkbj()
            if bj:
                break        
        
        #Each round show the current cards of each player
        showcards()
        for p in players[1:]:
            #Ask each player what they want to do and then react
            if p.out == False:
                print(f"{p.name}, what would you like to do?")
                action = 'A'
                #Surrender and Doubling Down are only allowed in the first round
                if round == 1:
                    while not action[0].upper() in ("H","S","D","U"):
                        #You can't double down without enough chips
                        if p.chips >= p.bet:
                            action = input("(H)it, (S)tand, (D)ouble down or S(U)rrender  \n")
                        else:
                            action = input("(H)it, (S)tand or S(U)rrender \n")    
                else:
                    while not action[0].upper() in ("H","S"):
                        action = input("(H)it or (S)tand  ")
                #Hit
                if action == 'H':
                    p.add_card(deck.deal_one())
                    showcards()
                #Stand
                elif action == 'S':
                    p.out = True
                    n_out += 1
                #Double down ends the game for this player, no more cards
                elif action == 'D':
                    p.add_card(deck.deal_one())
                    p.place_bet(p.bet)     
                    p.out = True
                    n_out += 1
                    showcards()
                #Surrender: Get half of your bet back, out of the loop
                elif action == 'U':
                    p.add_chips(p.bet//2)
                    p.bet = p.bet//2
                    p.out = True
                    p.bust = True
                    n_out += 1
                
                #Take all players out of the loop who have busted or reached exactly 21
                if p.total_value > 21:
                    print(f"{p.name} busted!")
                    p.out = True
                    n_out += 1
                    p.bust = True
                elif p.total_value == 21:
                    print(f"{p.name} has 21")
                    p.out = True
                    n_out += 1
                
        round += 1
    
    #This block comes when all players are finished and nobody had blackjack
    if bj == False:
        #Unnecessary if (should always be true here), but might help in case of bugs
        if n_out == len(players)-1:
            clear_output()
            print("All players are done. Dealer draws now:")
        #The dealer draws now until hitting 17 or bust
        while True:
            dealer.reveal()
            if dealer.total_value <= dealer.hit_until:
                print("Dealer hits.")
                dealer.add_card(deck.deal_one())
            elif dealer.total_value <= 21:
                print("Dealer stands.")
                break
            if dealer.total_value > 21:
                print("Dealer busts.")
                dealer.bust = True
                break
        print("Final table:")
        showfinal()

        #Now round up all values and distribute wins/losses
        roundup()
        
    #Below happens after each game
    #Automatically remove players with 0 chips from the game
    for p in players[1:]:
        if p.chips <= 0:
            print(f"{p.name} has no chips left and is leaving the game!")
            players.remove(p)
    
    #If there are no players left, end the program
    if len(players) < 2:
        print("No players left, game over")
        break
    else:
    #If there are still players left, ask the user
        l = "A"
        while not l[0].upper() in ("Y","N"): 
            l = input("Another game (y/n)?")
        if l.upper() == "Y":
            game += 1
        elif l.upper() == "N":
            print("Thank you for playing!")
            break
            

